In [1]:
# 라이브러리 불러오기
import pandas as pd

from konlpy.tag import Mecab
import numpy as np
from collections import Counter

import tensorflow as tf
import os

import matplotlib.pyplot as plt

In [3]:
# 데이터를 읽어봅시다. 
train_data = pd.read_table('data/ratings_train.txt')
test_data = pd.read_table('data/ratings_test.txt')

train_data.head()

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [5]:
tokenizer = Mecab()
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']

def load_data(train_data, test_data, num_words=10000):
  train_data.drop_duplicates(subset=['document'], inplace=True)  # 데이터 중복 제거
  train_data = train_data.dropna(how = 'any')  # NaN 결측치 제거
  test_data.drop_duplicates(subset=['document'], inplace=True)  # 데이터 중복 제거
  test_data = test_data.dropna(how = 'any')  # NaN 결측치 제거
  
  X_train = []  # 학습용 데이터
  for sentence in train_data['document']:
      temp_X = tokenizer.morphs(sentence) # 토큰화
      temp_X = [word for word in temp_X if not word in stopwords] # 불용어 제거
      X_train.append(temp_X)  # 단어 저장

  X_test = []  # 테스트용 데이터
  for sentence in test_data['document']:
      temp_X = tokenizer.morphs(sentence) # 토큰화
      temp_X = [word for word in temp_X if not word in stopwords] # 불용어 제거
      X_test.append(temp_X)  # 단어 저장
  
  words = np.concatenate(X_train).tolist()  # array 합치고 list로 변환
  counter = Counter(words)  # list의 요소 개수 세기 => 딕셔너리 {단어: 개수} 형태로 반환
  counter = counter.most_common(10000-4)  # 최빈값 (10000-4)개 => 가장 마지막 4개를 제외한 나머지
  vocab = ['', '', '', ''] + [key for key, _ in counter]  # 단어 저장 list # , , , 
  word_to_index = {word:index for index, word in enumerate(vocab)}  # 딕셔너리 {단어: 인덱스} 형태
  
  # 중첩 함수
  def wordlist_to_indexlist(wordlist):
      return [word_to_index[word] if word in word_to_index else word_to_index[''] for word in wordlist]
      
  # list(map(함수, 리스트)): 리스트의 모든 요소를 지정된 함수로 처리한 결과를 리스트로 만듦
  X_train = list(map(wordlist_to_indexlist, X_train))
  X_test = list(map(wordlist_to_indexlist, X_test))
      
  return X_train, np.array(list(train_data['label'])), X_test, np.array(list(test_data['label'])), word_to_index
     

In [6]:
X_train, y_train, X_test, y_test, word_to_index = load_data(train_data, test_data) 

In [7]:
index_to_word = {index:word for word, index in word_to_index.items()}

In [8]:
# 문장 1개를 활용할 딕셔너리와 함께 주면, 단어 인덱스 리스트 벡터로 변환해 주는 함수입니다. 
# 단, 모든 문장은 로 시작하는 것으로 합니다. 
def get_encoded_sentence(sentence, word_to_index):
    return [word_to_index['']]+[word_to_index[word] if word in word_to_index else word_to_index[''] for word in sentence.split()]

# 여러 개의 문장 리스트를 한꺼번에 단어 인덱스 리스트 벡터로 encode해 주는 함수입니다. 
def get_encoded_sentences(sentences, word_to_index):
    return [get_encoded_sentence(sentence, word_to_index) for sentence in sentences]

# 숫자 벡터로 encode된 문장을 원래대로 decode하는 함수입니다. 
def get_decoded_sentence(encoded_sentence, index_to_word):
    return ' '.join(index_to_word[index] if index in index_to_word else '' for index in encoded_sentence[1:])  #[1:]를 통해 를 제외

# 여러 개의 숫자 벡터로 encode된 문장을 한꺼번에 원래대로 decode하는 함수입니다. 
def get_decoded_sentences(encoded_sentences, index_to_word):
    return [get_decoded_sentence(encoded_sentence, index_to_word) for encoded_sentence in encoded_sentences]

In [9]:

print(X_train[0])  # 1번째 리뷰데이터
print('라벨: ', y_train[0])  # 1번째 리뷰데이터의 라벨

print('1번째 리뷰 문장 길이: ', len(X_train[0]))
print('2번째 리뷰 문장 길이: ', len(X_train[1]))
     

[32, 74, 919, 4, 4, 39, 228, 20, 33, 748]
라벨:  0
1번째 리뷰 문장 길이:  10
2번째 리뷰 문장 길이:  17


In [10]:
total_data_text = list(X_train) + list(X_test)
# 텍스트데이터 문장길이의 리스트를 생성한 후
num_tokens = [len(tokens) for tokens in total_data_text]
num_tokens = np.array(num_tokens)
# 문장길이의 평균값, 최대값, 표준편차를 계산해 본다. 
print('문장길이 평균 : ', np.mean(num_tokens))
print('문장길이 최대 : ', np.max(num_tokens))
print('문장길이 표준편차 : ', np.std(num_tokens))

# 예를들어, 최대 길이를 (평균 + 2*표준편차)로 한다면,  
max_tokens = np.mean(num_tokens) + 2 * np.std(num_tokens)
maxlen = int(max_tokens)
print('pad_sequences maxlen : ', maxlen)
print('전체 문장의 {}%가 maxlen 설정값 이내에 포함됩니다. '.format(np.sum(num_tokens < max_tokens) / len(num_tokens)))

문장길이 평균 :  15.96940191154864
문장길이 최대 :  116
문장길이 표준편차 :  12.843571191092
pad_sequences maxlen :  41
전체 문장의 0.9342988343341575%가 maxlen 설정값 이내에 포함됩니다. 


In [11]:
# 짧은 문장 앞(pre)에 패딩 추가. post보다 pre가 효율이 더 좋음!
X_train = tf.keras.preprocessing.sequence.pad_sequences(X_train,
                                                        value=word_to_index[""],
                                                        padding='pre', # 혹은 'post'
                                                        maxlen=maxlen)

X_test = tf.keras.preprocessing.sequence.pad_sequences(X_test,
                                                       value=word_to_index[""],
                                                       padding='pre', # 혹은 'post'
                                                       maxlen=maxlen)

print(X_train.shape)  # (146182, 41)
print(X_test.shape)  # (49157, 41)

(146182, 41)
(49157, 41)


In [13]:

vocab_size = len(word_to_index)  # 어휘 사전의 크기: 10000개 단어
word_vector_dim = 200   # 단어 하나를 표현하는 임베딩 벡터의 차원 수입니다. (변경 가능한 하이퍼 파라미터)
     

In [14]:

# 모델 1: RNN 

model_lstm = tf.keras.Sequential()
model_lstm.add(tf.keras.layers.Embedding(vocab_size, word_vector_dim, input_shape=(None,)))
model_lstm.add(tf.keras.layers.LSTM(8))   # 가장 널리 쓰이는 RNN인 LSTM 레이어를 사용하였습니다. 이때 LSTM state 벡터의 차원수는 8로 하였습니다. (변경 가능)
model_lstm.add(tf.keras.layers.Dense(8, activation='relu'))
model_lstm.add(tf.keras.layers.Dense(1, activation='sigmoid'))  # 최종 출력은 긍정/부정을 나타내는 1dim 입니다.

model_lstm.summary()
     

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 200)         1999400   
_________________________________________________________________
lstm (LSTM)                  (None, 8)                 6688      
_________________________________________________________________
dense (Dense)                (None, 8)                 72        
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 9         
Total params: 2,006,169
Trainable params: 2,006,169
Non-trainable params: 0
_________________________________________________________________


In [15]:
# 모델 2: 1-D CNN

model_1d_cnn = tf.keras.Sequential()
model_1d_cnn.add(tf.keras.layers.Embedding(vocab_size, word_vector_dim, input_shape=(None,)))
model_1d_cnn.add(tf.keras.layers.Conv1D(16, 7, activation='relu'))
model_1d_cnn.add(tf.keras.layers.MaxPooling1D(5))
model_1d_cnn.add(tf.keras.layers.Conv1D(16, 7, activation='relu'))
model_1d_cnn.add(tf.keras.layers.GlobalMaxPooling1D())
model_1d_cnn.add(tf.keras.layers.Dense(8, activation='relu'))
model_1d_cnn.add(tf.keras.layers.Dense(1, activation='sigmoid'))  # 최종 출력은 긍정/부정을 나타내는 1dim 입니다.

model_1d_cnn.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 200)         1999400   
_________________________________________________________________
conv1d (Conv1D)              (None, None, 16)          22416     
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, None, 16)          0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, None, 16)          1808      
_________________________________________________________________
global_max_pooling1d (Global (None, 16)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 8)                 136       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                

In [16]:
# 모델 3: GlobalMaxPooling1D() 레이어 1개만 사용

model_gmp1d = tf.keras.Sequential()
model_gmp1d.add(tf.keras.layers.Embedding(vocab_size, word_vector_dim, input_shape=(None,)))
model_gmp1d.add(tf.keras.layers.GlobalMaxPooling1D())
model_gmp1d.add(tf.keras.layers.Dense(8, activation='relu'))
model_gmp1d.add(tf.keras.layers.Dense(1, activation='sigmoid'))  # 최종 출력은 긍정/부정을 나타내는 1dim 입니다.

model_gmp1d.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, None, 200)         1999400   
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 200)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 8)                 1608      
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 9         
Total params: 2,001,017
Trainable params: 2,001,017
Non-trainable params: 0
_________________________________________________________________


In [17]:

# Train : Validation : Test 을 일반적으로 6 : 2 : 2로 이용

# 일반적으로 전체 데이터 중 80%를 학습으로, 20%를 검증으로 사용하는 것이 좋다고 한다.
# validation set (총 데이터 개수 * 0.2)건 분리
x_val_len = int(len(X_train)*0.2)
y_val_len = int(len(y_train)*0.2)

x_val = X_train[:x_val_len]   
y_val = y_train[:y_val_len]

print(x_val.shape)  # (29236, 41)
print(y_val.shape)  # (29236,)

# validation set을 제외한 나머지 (총 데이터 개수 * 0.8)건
partial_x_train = X_train[x_val_len:]  
partial_y_train = y_train[y_val_len:]

print(partial_x_train.shape)  # (116946, 41)
print(partial_y_train.shape)  # (116946,)
     

(29236, 41)
(29236,)
(116946, 41)
(116946,)


In [19]:

from keras.callbacks import EarlyStopping

# monitor='val_loss': validation set 의 loss 를 monitoring 한다.
# mode='min': loss 의 경우, performance measure가 최소화 시키는 방향으로 training 이 진행되므로 min 을 지정한다. => keras에서 알아서 적절한 epoch에서 training을 멈춘다.
# verbose=1: 언제 keras 에서 training 을 멈추었는지를 화면에 출력할 수 있다.
# patience=5: patience 는 성능이 증가하지 않는 epoch 을 몇 번이나 허용할 것인가를 주관적 기준으로 정의한다. 성능이 증가하지 않는다고 바로 멈추는 것은 효과적이지 않을 수 있기 때문이다. 사용한 데이터와 모델의 설계에 따라 최적의 값이 바뀔 수 있다. 
early_stopping = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)
     

In [24]:

model_lstm.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
              
epochs=5  # 몇 epoch를 훈련하면 좋을지 결과를 보면서 바꾸어 봅시다. 

history_lstm = model_lstm.fit(partial_x_train,
                    partial_y_train,
                    epochs=epochs,
                    batch_size=512,
                    validation_data=(x_val, y_val),
                    verbose=1,
                    callbacks=[early_stopping])
     

Epoch 1/5
229/229 [==============================] - 3s 8ms/step - loss: 0.1447 - accuracy: 0.9458 - val_loss: 0.4711 - val_accuracy: 0.8479
Epoch 2/5
229/229 [==============================] - 2s 7ms/step - loss: 0.1295 - accuracy: 0.9526 - val_loss: 0.4949 - val_accuracy: 0.8446
Epoch 3/5
229/229 [==============================] - 2s 7ms/step - loss: 0.1194 - accuracy: 0.9568 - val_loss: 0.5408 - val_accuracy: 0.8438
Epoch 4/5
229/229 [==============================] - 2s 7ms/step - loss: 0.1113 - accuracy: 0.9606 - val_loss: 0.5564 - val_accuracy: 0.8405
Epoch 5/5
229/229 [==============================] - 2s 7ms/step - loss: 0.1047 - accuracy: 0.9638 - val_loss: 0.5836 - val_accuracy: 0.8399


In [23]:
# 모델 평가
results_lstm = model_lstm.evaluate(X_test,  y_test, verbose=2)  # (loss, accuracy)

print(results_lstm)  # (0.3667, 0.8479)
     

1537/1537 - 3s - loss: 0.4525 - accuracy: 0.8424
[0.45247581601142883, 0.842382550239563]


In [25]:
model_1d_cnn.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
              
epochs=5  # 몇 epoch를 훈련하면 좋을지 결과를 보면서 바꾸어 봅시다. 

history_1d_cnn = model_1d_cnn.fit(partial_x_train,
                    partial_y_train,
                    epochs=epochs,
                    batch_size=512,
                    validation_data=(x_val, y_val),
                    verbose=1,
                    callbacks=[early_stopping])

Epoch 1/5
229/229 [==============================] - 15s 8ms/step - loss: 0.4306 - accuracy: 0.8000 - val_loss: 0.3387 - val_accuracy: 0.8522
Epoch 2/5
229/229 [==============================] - 1s 7ms/step - loss: 0.2964 - accuracy: 0.8761 - val_loss: 0.3256 - val_accuracy: 0.8601
Epoch 3/5
229/229 [==============================] - 1s 7ms/step - loss: 0.2291 - accuracy: 0.9112 - val_loss: 0.3428 - val_accuracy: 0.8578
Epoch 4/5
229/229 [==============================] - 2s 7ms/step - loss: 0.1553 - accuracy: 0.9441 - val_loss: 0.3899 - val_accuracy: 0.8526
Epoch 5/5
229/229 [==============================] - 1s 6ms/step - loss: 0.0949 - accuracy: 0.9683 - val_loss: 0.4635 - val_accuracy: 0.8491


In [26]:

# 모델 평가
results_1d_cnn = model_1d_cnn.evaluate(X_test,  y_test, verbose=2)  # (loss, accuracy)

print(results_1d_cnn)  # (0.4554, 0.8424)
     

1537/1537 - 3s - loss: 0.4873 - accuracy: 0.8424
[0.48730096220970154, 0.8424232602119446]


In [27]:

model_gmp1d.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
              
epochs=5  # 몇 epoch를 훈련하면 좋을지 결과를 보면서 바꾸어 봅시다. 

history_gmp1d = model_gmp1d.fit(partial_x_train,
                    partial_y_train,
                    epochs=epochs,
                    batch_size=512,
                    validation_data=(x_val, y_val),
                    verbose=1,
                    callbacks=[early_stopping])
     

Epoch 1/5
229/229 [==============================] - 2s 5ms/step - loss: 0.4750 - accuracy: 0.7971 - val_loss: 0.3474 - val_accuracy: 0.8485
Epoch 2/5
229/229 [==============================] - 1s 5ms/step - loss: 0.3144 - accuracy: 0.8675 - val_loss: 0.3339 - val_accuracy: 0.8543
Epoch 3/5
229/229 [==============================] - 1s 4ms/step - loss: 0.2670 - accuracy: 0.8916 - val_loss: 0.3389 - val_accuracy: 0.8562
Epoch 4/5
229/229 [==============================] - 1s 4ms/step - loss: 0.2269 - accuracy: 0.9109 - val_loss: 0.3536 - val_accuracy: 0.8563
Epoch 5/5
229/229 [==============================] - 1s 5ms/step - loss: 0.1871 - accuracy: 0.9305 - val_loss: 0.3772 - val_accuracy: 0.8544


In [28]:

# 모델 평가
results_gmp1d = model_gmp1d.evaluate(X_test,  y_test, verbose=2)  # (loss, accuracy)

print(results_gmp1d)  # (0.3834, 0.8461)
     

1537/1537 - 2s - loss: 0.3893 - accuracy: 0.8476
[0.38934779167175293, 0.8475903868675232]


출처 : https://github.com/HRPzz/AIFFEL/blob/main/EXPLORATION/Node_06/%5BE-06%5D%20Naver_movie_sentiment_analysis.ipynb

과제를 보고서 양이 너무 많아서 손도 못대다가, 루브릭 기준으로 하나라도 충족시켜보자는 생각에 무작정 찾아서 그대로 복붙을 하면서 대충 어떤식으로 진행되는지 흐름이라도 파악해보려고 했다.

모델들을 각자 가져와서 똑같이 훈련을 해도 결과값이 다 유의미하게 달랐고, 특히 train test 셋을 8:2로 기존에 알고있었는데, cs231n 강의에서 배운 validation 과정을 코드에 적용시켜 결과적으론 8:2지만 (6:2):2 로 결과값을 확인하는게 참신했다.